### Atención global

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Definir el módulo de atención
class Attention(nn.Module):
    def __init__(self, hidden_size, dropout=0.1):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, hidden, encoder_outputs, mask=None):
        """
        hidden: [batch_size, hidden_size]
        encoder_outputs: [src_len, batch_size, hidden_size]
        mask: [batch_size, src_len] (opcional)
        """
        src_len = encoder_outputs.size(0)
        batch_size = encoder_outputs.size(1)

        # Expand hidden to [batch_size, src_len, hidden_size]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch_size, src_len, hidden_size]

        # Concatenar y aplicar capa lineal y tanh
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, src_len, hidden_size]

        # Calcular puntuaciones de atención
        energy = energy.permute(0, 2, 1)  # [batch_size, hidden_size, src_len]
        v = self.v.repeat(batch_size, 1).unsqueeze(1)  # [batch_size, 1, hidden_size]
        attention = torch.bmm(v, energy).squeeze(1)  # [batch_size, src_len]

        if mask is not None:
            attention = attention.masked_fill(mask == 0, -1e10)

        attention = self.softmax(attention)
        attention = self.dropout(attention)

        return attention  # [batch_size, src_len]

# Definir el Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, num_layers=1, dropout=0.1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=num_layers, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        """
        src: [src_len, batch_size]
        src_lengths: [batch_size]
        """
        embedded = self.dropout(self.embedding(src))  # [src_len, batch_size, embed_dim]
        packed = nn.utils.rnn.pack_padded_sequence(embedded, src_lengths.cpu(), enforce_sorted=False)
        outputs, hidden = self.gru(packed)  # outputs: [src_len, batch, hidden_dim]
        src_len = src.size(0)  # Obtener el src_len original
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, total_length=src_len)
        return outputs, hidden  # outputs: [src_len, batch_size, hidden_dim]

# Definir el Decoder con Atención
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, attention, num_layers=1, dropout=0.1):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.attention = attention
        self.gru = nn.GRU(embed_dim + hidden_dim, hidden_dim, num_layers=num_layers, dropout=dropout if num_layers > 1 else 0)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs, mask=None):
        """
        input: [batch_size]
        hidden: [num_layers, batch_size, hidden_dim]
        encoder_outputs: [src_len, batch_size, hidden_dim]
        mask: [batch_size, src_len] (opcional)
        """
        input = input.unsqueeze(0)  # [1, batch_size]
        embedded = self.dropout(self.embedding(input))  # [1, batch_size, embed_dim]

        # Atención
        hidden_last = hidden[-1]  # [batch_size, hidden_dim]
        attn_weights = self.attention(hidden_last, encoder_outputs, mask)  # [batch_size, src_len]

        # Contexto
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch_size, src_len, hidden_dim]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)  # [batch_size, 1, hidden_dim]
        context = context.permute(1, 0, 2)  # [1, batch_size, hidden_dim]

        # Concatenar embedding y contexto
        rnn_input = torch.cat((embedded, context), dim=2)  # [1, batch_size, embed_dim + hidden_dim]

        # Decodificar
        output, hidden = self.gru(rnn_input, hidden)  # output: [1, batch_size, hidden_dim]

        # Concatenar output y contexto para predecir
        output = output.squeeze(0)  # [batch_size, hidden_dim]
        context = context.squeeze(0)  # [batch_size, hidden_dim]
        output = self.fc_out(torch.cat((output, context), dim=1))  # [batch_size, output_dim]

        return output, hidden, attn_weights  # output: [batch_size, output_dim]

# Definir el modelo Encoder-Decoder con Atención
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def create_mask(self, src, src_lengths):
        """
        Crear máscara para ignorar los padding en la atención
        src: [src_len, batch_size]
        src_lengths: [batch_size]
        """
        batch_size = src.shape[1]
        src_len = src.shape[0]
        # Crear una matriz [batch_size, src_len] donde cada posición es True si está dentro de la longitud real
        mask = torch.arange(src_len).unsqueeze(0).to(self.device) < src_lengths.unsqueeze(1)
        return mask  # [batch_size, src_len]

    def forward(self, src, src_lengths, trg, teacher_forcing_ratio=0.5):
        """
        src: [src_len, batch_size]
        src_lengths: [batch_size]
        trg: [trg_len, batch_size]
        teacher_forcing_ratio: probabilidad de usar el token real como siguiente input
        """
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        output_dim = self.decoder.output_dim

        # Tensor para guardar las predicciones
        outputs = torch.zeros(trg_len, batch_size, output_dim).to(self.device)

        # Codificar
        encoder_outputs, hidden = self.encoder(src, src_lengths)

        # Inicializar entrada del decoder (generalmente <sos>)
        input = trg[0, :]  # [batch_size]

        # Crear máscara
        mask = self.create_mask(src, src_lengths)  # [batch_size, src_len]

        for t in range(1, trg_len):
            output, hidden, attn_weights = self.decoder(input, hidden, encoder_outputs, mask)
            outputs[t] = output

            # Decidir si usar teacher forcing
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)  # [batch_size]

            input = trg[t] if teacher_force else top1

        return outputs

# Configuración de dimensiones y ejemplo de uso
INPUT_DIM = 1000   # Tamaño del vocabulario de entrada
OUTPUT_DIM = 1000  # Tamaño del vocabulario de salida
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Seleccionar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instanciar componentes
attn = Attention(HIDDEN_DIM, dropout=DEC_DROPOUT)
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HIDDEN_DIM, num_layers=N_LAYERS, dropout=ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HIDDEN_DIM, attn, num_layers=N_LAYERS, dropout=DEC_DROPOUT)

# Instanciar el modelo Seq2Seq
model = Seq2Seq(encoder, decoder, device).to(device)

# Inicializar pesos
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Asumiendo que 0 es el padding

# Ejemplo de datos (utilizar índices de vocabulario reales en aplicaciones reales)
src = torch.randint(1, INPUT_DIM, (10, 32)).to(device)  # src_len=10, batch_size=32
trg = torch.randint(1, OUTPUT_DIM, (20, 32)).to(device)  # trg_len=20, batch_size=32
src_lengths = torch.randint(5, 10, (32,)).to(device)  # Longitudes variables entre 5 y 9

# Hacer una pasada hacia adelante
output = model(src, src_lengths, trg, teacher_forcing_ratio=0.75)
print("Forma de la salida:", output.shape)  # Esperado: (trg_len, batch_size, output_dim)


### Redes de memoria extremo a extremo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MemoryNetwork(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hops=3, dropout=0.2):
        """
        Modelo de Red de Memoria con múltiples hops.

        Args:
            vocab_size (int): Tamaño del vocabulario.
            embedding_dim (int): Dimensionalidad de los embeddings.
            hops (int, opcional): Número de hops de atención. Por defecto es 3.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.2.
        """
        super(MemoryNetwork, self).__init__()
        self.hops = hops
        self.embedding_dim = embedding_dim

        # Embeddings para las sentencias y las preguntas
        self.embeddings_A = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings_C = nn.ModuleList([nn.Embedding(vocab_size, embedding_dim) for _ in range(hops)])

        # Red para combinar los embeddings de la pregunta
        self.question_combine = nn.Linear(embedding_dim, embedding_dim)
        self.relu = nn.ReLU()

        # Capa de salida
        self.fc = nn.Linear(embedding_dim, vocab_size)

        # Dropout para regularización
        self.dropout = nn.Dropout(dropout)

        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        """
        Inicializa los pesos de las capas de embedding y lineales.
        """
        nn.init.xavier_uniform_(self.embeddings_A.weight)
        for embed in self.embeddings_C:
            nn.init.xavier_uniform_(embed.weight)
        nn.init.xavier_uniform_(self.question_combine.weight)
        nn.init.zeros_(self.question_combine.bias)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, sentences, question, mask=None):
        """
        Propagación hacia adelante del modelo de Red de Memoria.

        Args:
            sentences (Tensor): Tensores de sentencias de forma (batch_size, num_sentences, sentence_length).
            question (Tensor): Tensores de preguntas de forma (batch_size, question_length).
            mask (Tensor, opcional): Máscara para las sentencias de forma (batch_size, num_sentences).

        Returns:
            Tensor: Log probabilidades de las respuestas de forma (batch_size, vocab_size).
        """
        batch_size, num_sentences, sentence_length = sentences.size()
        _, question_length = question.size()

        # Paso 1: Obtener embeddings de las preguntas
        # Embeddings_A se utiliza para las preguntas
        q_embeddings = self.embeddings_A(question)  # (batch_size, question_length, embedding_dim)
        q_embeddings = q_embeddings.mean(dim=1)  # (batch_size, embedding_dim)
        q_embeddings = self.dropout(q_embeddings)
        q = self.relu(self.question_combine(q_embeddings))  # (batch_size, embedding_dim)

        for hop in range(self.hops):
            # Paso 2: Obtener embeddings de las sentencias
            m = self.embeddings_A(sentences.view(-1, sentence_length))  # (batch_size * num_sentences, sentence_length, embedding_dim)
            m = m.mean(dim=1)  # (batch_size * num_sentences, embedding_dim)
            m = m.view(batch_size, num_sentences, self.embedding_dim)  # (batch_size, num_sentences, embedding_dim)

            # Embeddings_C específicos para cada hop
            c = self.embeddings_C[hop](sentences.view(-1, sentence_length))  # (batch_size * num_sentences, sentence_length, embedding_dim)
            c = c.mean(dim=1)  # (batch_size * num_sentences, embedding_dim)
            c = c.view(batch_size, num_sentences, self.embedding_dim)  # (batch_size, num_sentences, embedding_dim)

            # Paso 3: Cálculo de pesos de atención
            # Similaridad entre la pregunta y cada sentencia
            attn_scores = torch.bmm(m, q.unsqueeze(2)).squeeze(2)  # (batch_size, num_sentences)

            if mask is not None:
                attn_scores = attn_scores.masked_fill(mask == 0, -1e10)

            attn_weights = F.softmax(attn_scores, dim=1)  # (batch_size, num_sentences)
            attn_weights = self.dropout(attn_weights)

            # Paso 4: Suma ponderada de las sentencias
            o = torch.bmm(attn_weights.unsqueeze(1), c).squeeze(1)  # (batch_size, embedding_dim)

            # Integración de la memoria con el estado de la pregunta
            q = q + o  # (batch_size, embedding_dim)
            q = self.dropout(q)

        # Paso 5: Predicción final
        output = self.fc(q)  # (batch_size, vocab_size)
        return F.log_softmax(output, dim=1)

# Configuración de dimensiones y ejemplo de uso
if __name__ == "__main__":
    # Parámetros del modelo
    vocab_size = 1000      # Tamaño del vocabulario
    embedding_dim = 128    # Dimensionalidad de los embeddings
    hops = 3               # Número de hops de atención
    dropout = 0.3          # Tasa de dropout

    # Crear instancia del modelo
    model = MemoryNetwork(vocab_size, embedding_dim, hops, dropout)

    # Seleccionar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Ejemplo de datos
    batch_size = 32
    num_sentences = 10
    sentence_length = 15
    question_length = 5

    # Sentencias: (batch_size, num_sentences, sentence_length)
    sentences = torch.randint(0, vocab_size, (batch_size, num_sentences, sentence_length)).to(device)

    # Preguntas: (batch_size, question_length)
    question = torch.randint(0, vocab_size, (batch_size, question_length)).to(device)

    # Máscara opcional (por ejemplo, para padding)
    mask = torch.ones(batch_size, num_sentences).to(device)  # Aquí no hay padding

    # Predicción
    output = model(sentences, question, mask)
    print("Predicción de respuesta:", output.argmax(dim=1))  # (batch_size)
    print("Forma de la salida:", output.shape)  # (batch_size, vocab_size)


Las **memory networks** son una clase de modelos de aprendizaje profundo diseñados para tareas de procesamiento de lenguaje natural que requieren razonamiento a múltiples pasos o "hops" sobre una memoria externa. Estas memorias están compuestas por sentencias o fragmentos de texto que el modelo puede consultar iterativamente para extraer información relevante.

**Características clave:**
- **Memoria externa:** Almacena información que el modelo puede consultar.
- **Múltiples hops de atención:** Permite al modelo realizar múltiples iteraciones de atención para refinar su comprensión y respuesta.
- **Actualización del estado:** Después de cada hop, el estado interno del modelo se actualiza incorporando la información extraída de la memoria.

#### **Mecanismo de atención de Bahdanau**
El mecanismo de **atención de Bahdanau**, también conocido como **atención aditiva**, es una técnica diseñada para mejorar los modelos de secuencia a secuencia (Seq2Seq) al permitir que el decodificador se enfoque en diferentes partes de la secuencia de entrada en cada paso de generación.

**Características clave:**
- **Atención puntual:** En cada paso de generación, el decodificador calcula pesos de atención sobre todas las posiciones de la secuencia de entrada.
- **Contexto dinámico:** Genera un vector de contexto dinámico que es una combinación ponderada de las representaciones de la entrada, guiado por los pesos de atención.
- **Un solo paso de atención:** Generalmente, realiza un solo cálculo de atención por paso de generación.


A continuación, comparamos ambos enfoques en términos de sus mecanismos y cómo se reflejan en el código de `MemoryNetwork`.

#### **a. Estructura de la memoria**

- **Memory networks:**
  - **Memoria estructurada:** Las memorias están organizadas en sentencias o hechos individuales.
  - **Múltiples hops:** Permiten múltiples iteraciones para refinar la atención sobre la memoria.
  
- **Bahdanau attention:**
  - **Memoria implícita:** La memoria está implícita en las salidas del codificador.
  - **Atención singular:** Realiza una única atención por paso de decodificación.

#### **b. Mecanismo de atención**

- **Memory networks:**
  - **Atención iterativa:** En cada hop, el modelo recalcula los pesos de atención y actualiza su estado interno.
  - **Integración de la memoria:** El estado interno se actualiza agregando la información extraída de la memoria en cada hop.
  
- **Bahdanau attention:**
  - **Atención adicional por paso:** En cada paso de decodificación, se calcula una nueva distribución de atención sobre las entradas.
  - **Contexto basado en estado actual:** El vector de contexto se basa únicamente en el estado actual del decodificador.

#### **c. Actualización del estado interno**

- **Memory networks:**
  - **Estado refinado:** El estado interno (`u` en el código) se actualiza iterativamente con la información extraída de la memoria.
  
- **Bahdanau attention:**
  - **Estado independiente por paso:** Cada paso de atención depende del estado actual del decodificador, pero no se mantiene un estado acumulativo a través de múltiples atenciones.

#### **d. Aplicación en el código proporcionado**



#### **MemoryNetwork**

```python
class MemoryNetwork(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hops=3):
        super(MemoryNetwork, self).__init__()
        self.hops = hops
        self.embeddings_A = nn.Embedding(vocab_size, embedding_dim)
        self.embeddings_C = nn.ModuleList([nn.Embedding(vocab_size, embedding_dim) for _ in range(hops)])
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, sentences, question):
        # Paso 1: Obtener embedding de la pregunta
        u = self.embeddings_A(question).mean(dim=1)  # (batch, embedding_dim)
        
        for hop in range(self.hops):
            # Paso 2: Embedding de las sentencias (memoria)
            m = self.embeddings_A(sentences)  # (batch, num_sentences, embedding_dim)
            c = self.embeddings_C[hop](sentences)  # (batch, num_sentences, embedding_dim)

            # Paso 3: Cálculo de pesos de atención (similaridad entre pregunta y sentencias)
            p = F.softmax(torch.bmm(m, u.unsqueeze(2)).squeeze(2), dim=1)  # (batch, num_sentences)
            
            # Paso 4: Suma ponderada
            o = torch.bmm(p.unsqueeze(1), c).squeeze(1)  # (batch, embedding_dim)
            
            # Actualizar u para el siguiente salto
            u = u + o  # Integración de la memoria de salida con el estado de la pregunta

        # Paso 5: Predicción final
        output = self.fc(u)  # (batch, vocab_size)
        return F.log_softmax(output, dim=1)
```

**Análisis en el contexto de memory networks:**

1. **Memoria y hops:**
   - **`self.hops`:** Define el número de hops de atención, permitiendo múltiples iteraciones para refinar la atención sobre las sentencias.
   - **Embeddings múltiples (`embeddings_A` y `embeddings_C`):** Diferentes embeddings para cada hop permiten al modelo capturar diferentes aspectos de las sentencias en cada iteración.

2. **Iteraciones de atención:**
   - En cada hop, el modelo calcula una distribución de atención (`p`) sobre las sentencias basándose en la similitud entre la pregunta (`u`) y las sentencias (`m`).
   - El vector de estado interno (`u`) se actualiza agregando la información extraída (`o`), lo que permite al modelo acumular información relevante a lo largo de los hops.

3. **Predicción final:**
   - Después de los hops de atención, el estado interno refinado (`u`) se pasa a una capa lineal para generar la predicción final.

**Comparación con Bahdanau:**
- **Múltiples iteraciones vs. atención singular:** A diferencia del mecanismo de Bahdanau, que realiza una sola atención por paso de decodificación, `MemoryNetwork` permite múltiples iteraciones de atención, refinando continuamente el estado interno.
- **Actualización del estado:** `MemoryNetwork` mantiene y actualiza un estado interno acumulativo (`u`), mientras que Bahdanau no mantiene un estado acumulativo a través de las atenciones.

#### **Bahdanau Attention (conceptual)**

Para comparar, consideremos una implementación conceptual de Bahdanau Attention en un modelo Seq2Seq:

```python
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch_size, hidden_dim]
        # encoder_outputs: [src_len, batch_size, hidden_dim]
        
        src_len = encoder_outputs.size(0)
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # [batch_size, src_len, hidden_dim]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch_size, src_len, hidden_dim]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, src_len, hidden_dim]
        energy = energy.permute(0, 2, 1)  # [batch_size, hidden_dim, src_len]
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [batch_size, 1, hidden_dim]
        attention = torch.bmm(v, energy).squeeze(1)  # [batch_size, src_len]
        
        return F.softmax(attention, dim=1)  # [batch_size, src_len]
```

**Características en comparación:**

1. **Atención singular:**
   - En cada paso de decodificación, calcula una única distribución de atención sobre las salidas del codificador.

2. **No hay actualización acumulativa:**
   - No mantiene un estado interno que se actualiza iterativamente a través de múltiples atenciones.

#### **3. Implicaciones**

#### **a. Número de hops**

En el código de `MemoryNetwork`, el parámetro `hops` permite al modelo realizar múltiples iteraciones de atención sobre la memoria (sentencias). Esto es fundamental en las Memory Networks de Sukhbaatar, ya que cada hop puede enfocarse en diferentes aspectos o fragmentos de la memoria para refinar la respuesta.

En contraste, el mecanismo de Bahdanau generalmente realiza una única atención por paso de decodificación, enfocándose en diferentes partes de la entrada en cada paso de generación, pero sin iteraciones internas adicionales por paso.

#### **b. Actualización del estado interno**

En `MemoryNetwork`, el estado interno `u` se actualiza agregando el vector `o` obtenido de la atención en cada hop:

```python
u = u + o
```

Esto permite que el estado interno acumule información relevante de múltiples hops, mejorando la capacidad del modelo para razonar sobre la información almacenada en la memoria.

En Bahdanau Attention, el estado del decodificador (`hidden`) se utiliza directamente para calcular la atención en cada paso, pero no se actualiza de manera acumulativa a través de múltiples iteraciones de atención internas.

#### **c. Mecanismo de atención iterativo vs. singular**

El código de `MemoryNetwork` implementa un ciclo `for` sobre los hops:

```python
for hop in range(self.hops):
    # Cálculo de atención y actualización de u
```

Esto refleja el proceso iterativo de las Memory Networks, donde cada hop puede potencialmente extraer información adicional de la memoria, refinando el entendimiento de la pregunta.

Por otro lado, en Bahdanau Attention, no existe tal ciclo interno; la atención se calcula una vez por paso de decodificación.

#### **d. Integración de la memoria y la pregunta**

En `MemoryNetwork`, la pregunta (`u`) y las sentencias se combinan en cada hop para calcular los pesos de atención:

```python
p = F.softmax(torch.bmm(m, u.unsqueeze(2)).squeeze(2), dim=1)
```

Esto representa una interacción directa y repetitiva entre la pregunta y la memoria en múltiples hops.

En Bahdanau Attention, la interacción entre el estado del decodificador y las salidas del codificador se realiza una sola vez por paso de decodificación.

#### **e. Capas de embedding diferentes por hop**

El uso de `embeddings_C` como una lista de capas de embedding específicas para cada hop permite al modelo capturar diferentes representaciones de las sentencias en cada iteración. Esto añade flexibilidad y capacidad de razonamiento más profundo.

Bahdanau Attention no tiene una correspondencia directa para esto, ya que utiliza una única representación de las entradas en cada paso de atención.



### Sum Reader

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    """
    Mecanismo de atención para calcular la relevancia entre dos representaciones.
    """
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Parameter(torch.rand(hidden_dim))
        nn.init.xavier_uniform_(self.attn.weight)
        nn.init.xavier_uniform_(self.v.unsqueeze(0))

    def forward(self, hidden, encoder_outputs, mask=None):
        """
        Args:
            hidden: [batch_size, hidden_dim] - Estado oculto de la pregunta.
            encoder_outputs: [batch_size, seq_len, hidden_dim] - Salidas del documento.
            mask: [batch_size, seq_len] - Máscara para ignorar padding.
        
        Returns:
            attention_weights: [batch_size, seq_len] - Pesos de atención.
        """
        batch_size, seq_len, hidden_dim = encoder_outputs.size()
        
        # Expandir el estado oculto de la pregunta para concatenarlo con las salidas del documento
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # [batch_size, seq_len, hidden_dim]
        
        # Concatenar y aplicar capa lineal y tanh
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, seq_len, hidden_dim]
        
        # Calcular puntuaciones de atención
        energy = energy.view(-1, hidden_dim)  # [batch_size * seq_len, hidden_dim]
        v = self.v.repeat(batch_size, 1)  # [batch_size, hidden_dim]
        attention = torch.bmm(energy.view(batch_size, seq_len, hidden_dim), v.unsqueeze(2)).squeeze(2)  # [batch_size, seq_len]
        
        if mask is not None:
            attention = attention.masked_fill(mask == 0, -1e10)
        
        attention_weights = F.softmax(attention, dim=1)  # [batch_size, seq_len]
        return attention_weights

class SumReader(nn.Module):
    """
    Modelo SumReader mejorado para procesar documentos y preguntas,
    utilizando RNNs bidireccionales y un mecanismo de atención.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout=0.3):
        """
        Args:
            vocab_size (int): Tamaño del vocabulario.
            embedding_dim (int): Dimensionalidad de los embeddings.
            hidden_dim (int): Dimensionalidad de las capas ocultas de las RNNs.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.3.
        """
        super(SumReader, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        
        # RNNs bidireccionales para el documento y la pregunta
        self.document_rnn = nn.GRU(embedding_dim, hidden_dim, bidirectional=True, batch_first=True, dropout=dropout)
        self.question_rnn = nn.GRU(embedding_dim, hidden_dim, bidirectional=True, batch_first=True, dropout=dropout)
        
        # Capa de atención
        self.attention = Attention(hidden_dim * 2)
        
        # Capa lineal para la predicción final
        self.fc = nn.Linear(hidden_dim * 2, vocab_size)
        
        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        """
        Inicializa los pesos de las capas de embedding y lineales.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)

    def forward(self, document, question, doc_lengths, ques_lengths):
        """
        Propagación hacia adelante del modelo SumReader.
        
        Args:
            document (Tensor): [batch_size, doc_len] - Índices de palabras del documento.
            question (Tensor): [batch_size, ques_len] - Índices de palabras de la pregunta.
            doc_lengths (Tensor): [batch_size] - Longitudes de cada documento.
            ques_lengths (Tensor): [batch_size] - Longitudes de cada pregunta.
        
        Returns:
            output: [batch_size, vocab_size] - Log probabilidades de las respuestas.
        """
        # Paso 1: Embedding del documento y de la pregunta
        doc_embed = self.dropout(self.embedding(document))  # [batch_size, doc_len, embedding_dim]
        ques_embed = self.dropout(self.embedding(question)) # [batch_size, ques_len, embedding_dim]
        
        # Paso 2: Pasar el documento y la pregunta por las RNNs con packing
        # Document
        packed_doc = nn.utils.rnn.pack_padded_sequence(doc_embed, doc_lengths.cpu(), batch_first=True, enforce_sorted=False)
        doc_output, _ = self.document_rnn(packed_doc)   # [batch_size, doc_len, hidden_dim * 2]
        doc_output, _ = nn.utils.rnn.pad_packed_sequence(doc_output, batch_first=True)
        
        # Pregunta
        packed_ques = nn.utils.rnn.pack_padded_sequence(ques_embed, ques_lengths.cpu(), batch_first=True, enforce_sorted=False)
        ques_output, _ = self.question_rnn(packed_ques) # [batch_size, ques_len, hidden_dim * 2]
        ques_output, _ = nn.utils.rnn.pad_packed_sequence(ques_output, batch_first=True)
        
        # Representación de la pregunta como el último estado oculto de la RNN
        # Alternativamente, se puede usar una atención sobre la pregunta
        question_representation = torch.cat((ques_output[range(len(ques_output)), ques_lengths - 1, :hidden_dim],
                                             ques_output[range(len(ques_output)), ques_lengths - 1, hidden_dim:]), dim=1)  # [batch_size, hidden_dim * 2]
        
        # Paso 3: Calcular pesos de atención entre la pregunta y cada palabra del documento
        attention_weights = self.attention(question_representation, doc_output)  # [batch_size, doc_len]
        
        # Paso 4: Calcular el vector de contexto como suma ponderada de las representaciones del documento
        context = torch.bmm(attention_weights.unsqueeze(1), doc_output).squeeze(1)  # [batch_size, hidden_dim * 2]
        
        # Paso 5: Pasar el vector de contexto por una capa lineal para predecir la respuesta
        output = self.fc(context)  # [batch_size, vocab_size]
        
        return F.log_softmax(output, dim=1)  # [batch_size, vocab_size]

# Configuración del modelo
if __name__ == "__main__":
    # Parámetros del modelo
    vocab_size = 10000     # Tamaño del vocabulario
    embedding_dim = 300    # Dimensionalidad de los embeddings
    hidden_dim = 128       # Dimensionalidad oculta de las RNNs
    dropout = 0.3          # Tasa de dropout

    # Crear instancia del modelo
    model = SumReader(vocab_size, embedding_dim, hidden_dim, dropout)

    # Seleccionar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Ejemplo de datos
    batch_size = 2
    doc_len = 10
    ques_len = 5

    # Sentencias: [batch_size, doc_len]
    document = torch.randint(1, vocab_size, (batch_size, doc_len)).to(device)  # Asumiendo que 0 es <pad>
    # Preguntas: [batch_size, ques_len]
    question = torch.randint(1, vocab_size, (batch_size, ques_len)).to(device)   # Asumiendo que 0 es <pad>

    # Longitudes de las secuencias (sin padding)
    doc_lengths = torch.tensor([10, 8]).to(device)   # Ejemplo de longitudes
    ques_lengths = torch.tensor([5, 4]).to(device)  # Ejemplo de longitudes

    # Predicción de probabilidad de respuesta
    output = model(document, question, doc_lengths, ques_lengths)
    print("Probabilidad de respuesta:", output)
    print("Forma de la salida:", output.shape)  # [batch_size, vocab_size]


### Two-Way

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TwoWayAttention(nn.Module):
    """
    Mecanismo de atención bidireccional entre secuencias fuente y destino.

    Este módulo calcula atención desde la fuente hacia el destino y desde el destino hacia la fuente,
    combina las representaciones contextuales y produce logits para clasificación.
    """
    def __init__(self, hidden_dim, num_classes=3, dropout=0.3):
        """
        Args:
            hidden_dim (int): Dimensionalidad de los estados ocultos.
            num_classes (int, opcional): Número de clases de salida. Por defecto es 3.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.3.
        """
        super(TwoWayAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.dropout = nn.Dropout(dropout)
        
        # Capas lineales para proyectar las representaciones de atención
        self.attention_source = nn.Linear(hidden_dim, hidden_dim)
        self.attention_target = nn.Linear(hidden_dim, hidden_dim)
        
        # Función de activación
        self.activation = nn.ReLU()
        
        # Capas completamente conectadas para la clasificación
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, num_classes)
        
        # Inicialización de pesos
        self._init_weights()
    
    def _init_weights(self):
        """
        Inicializa los pesos de las capas lineales utilizando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.attention_source.weight)
        nn.init.constant_(self.attention_source.bias, 0)
        nn.init.xavier_uniform_(self.attention_target.weight)
        nn.init.constant_(self.attention_target.bias, 0)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.xavier_uniform_(self.output_layer.weight)
        nn.init.constant_(self.output_layer.bias, 0)
    
    def forward(self, source_states, target_states, source_mask=None, target_mask=None):
        """
        Propagación hacia adelante del mecanismo de atención bidireccional.

        Args:
            source_states (Tensor): Estados ocultos de la secuencia fuente [batch, src_len, hidden_dim]
            target_states (Tensor): Estados ocultos de la secuencia destino [batch, tgt_len, hidden_dim]
            source_mask (Tensor, opcional): Máscara para la secuencia fuente [batch, src_len]
            target_mask (Tensor, opcional): Máscara para la secuencia destino [batch, tgt_len]

        Returns:
            Tensor: Log probabilidades para cada clase [batch, num_classes]
        """
        # Paso 1: Obtener representaciones globales de la fuente y el destino
        # Usando mean pooling con máscara si está disponible
        if source_mask is not None:
            # Evitar división por cero
            src_lengths = source_mask.sum(dim=1, keepdim=True).clamp(min=1)
            source_rep = (source_states * source_mask.unsqueeze(2)).sum(dim=1) / src_lengths
        else:
            source_rep = source_states.mean(dim=1)  # [batch, hidden_dim]
        
        if target_mask is not None:
            tgt_lengths = target_mask.sum(dim=1, keepdim=True).clamp(min=1)
            target_rep = (target_states * target_mask.unsqueeze(2)).sum(dim=1) / tgt_lengths
        else:
            target_rep = target_states.mean(dim=1)  # [batch, hidden_dim]
        
        # Aplicar dropout a las representaciones
        source_rep = self.dropout(source_rep)
        target_rep = self.dropout(target_rep)
        
        # Paso 2: Proyectar las representaciones
        source_proj = self.attention_source(source_rep)  # [batch, hidden_dim]
        target_proj = self.attention_target(target_rep)  # [batch, hidden_dim]
        
        # Paso 3: Atención de fuente hacia destino
        # Calcular pesos de atención de destino basados en la fuente
        source_proj_expanded = source_proj.unsqueeze(2)  # [batch, hidden_dim, 1]
        attention_weights_target = torch.bmm(target_states, source_proj_expanded).squeeze(2)  # [batch, tgt_len]
        
        if target_mask is not None:
            attention_weights_target = attention_weights_target.masked_fill(target_mask == 0, -1e10)
        
        attention_weights_target = F.softmax(attention_weights_target, dim=1)  # [batch, tgt_len]
        attention_weights_target = self.dropout(attention_weights_target)
        
        # Vector de contexto para el destino
        target_context = torch.bmm(attention_weights_target.unsqueeze(1), target_states).squeeze(1)  # [batch, hidden_dim]
        
        # Paso 4: Atención de destino hacia fuente
        # Calcular pesos de atención de fuente basados en el destino
        target_proj_expanded = target_proj.unsqueeze(2)  # [batch, hidden_dim, 1]
        attention_weights_source = torch.bmm(source_states, target_proj_expanded).squeeze(2)  # [batch, src_len]
        
        if source_mask is not None:
            attention_weights_source = attention_weights_source.masked_fill(source_mask == 0, -1e10)
        
        attention_weights_source = F.softmax(attention_weights_source, dim=1)  # [batch, src_len]
        attention_weights_source = self.dropout(attention_weights_source)
        
        # Vector de contexto para la fuente
        source_context = torch.bmm(attention_weights_source.unsqueeze(1), source_states).squeeze(1)  # [batch, hidden_dim]
        
        # Paso 5: Concatenar los vectores de contexto
        combined_context = torch.cat([target_context, source_context], dim=1)  # [batch, hidden_dim * 2]
        combined_context = self.dropout(combined_context)
        
        # Paso 6: Pasar por una capa completamente conectada y aplicar activación
        combined_representation = self.activation(self.fc(combined_context))  # [batch, hidden_dim]
        combined_representation = self.dropout(combined_representation)
        
        # Paso 7: Capa de salida para clasificación
        output = self.output_layer(combined_representation)  # [batch, num_classes]
        
        # Retornar log probabilidades
        return F.log_softmax(output, dim=1)  # [batch, num_classes]

# Ejemplo de configuración del modelo
if __name__ == "__main__":
    # Parámetros del modelo
    hidden_dim = 128      # Dimensión oculta de los estados
    num_classes = 3       # Número de clases para clasificación
    dropout = 0.3         # Tasa de dropout
    
    # Crear instancia del modelo
    model = TwoWayAttention(hidden_dim, num_classes=num_classes, dropout=dropout)
    
    # Seleccionar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Ejemplo de datos de entrada
    batch_size = 4
    seq_len_source = 5
    seq_len_target = 7
    
    # Simulación de estados ocultos de source y target
    source_states = torch.randn(batch_size, seq_len_source, hidden_dim).to(device)
    target_states = torch.randn(batch_size, seq_len_target, hidden_dim).to(device)
    
    # Opcional: máscaras para source y target (1 indica válido, 0 indica padding)
    source_mask = torch.ones(batch_size, seq_len_source).to(device)  # Aquí no hay padding
    target_mask = torch.ones(batch_size, seq_len_target).to(device)  # Aquí no hay padding
    
    # Obtener salida del modelo
    output = model(source_states, target_states, source_mask=source_mask, target_mask=target_mask)
    print("Probabilidades de clasificación:", output)
    print("Forma de la salida:", output.shape)  # [batch, num_classes]


### Redes dinámicas de coatención

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Coattention(nn.Module):
    """
    Mecanismo de Co-Atención bidireccional entre secuencias fuente y destino.
    """
    def __init__(self, hidden_dim, dropout=0.3):
        """
        Args:
            hidden_dim (int): Dimensionalidad de los estados ocultos.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.3.
        """
        super(Coattention, self).__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout)
        
        # Capas lineales para proyectar las representaciones de atención
        self.attention_source = nn.Linear(hidden_dim, hidden_dim)
        self.attention_target = nn.Linear(hidden_dim, hidden_dim)
        
        # Función de activación
        self.activation = nn.ReLU()
        
        # Capa lineal para combinar las representaciones contextuales
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim)
        
    def forward(self, source_states, target_states, source_mask=None, target_mask=None):
        """
        Propagación hacia adelante del mecanismo de co-atención.
    
        Args:
            source_states (Tensor): Estados ocultos de la secuencia fuente [batch, src_len, hidden_dim]
            target_states (Tensor): Estados ocultos de la secuencia destino [batch, tgt_len, hidden_dim]
            source_mask (Tensor, opcional): Máscara para la secuencia fuente [batch, src_len]
            target_mask (Tensor, opcional): Máscara para la secuencia destino [batch, tgt_len]
    
        Returns:
            Tensor: Representación combinada [batch, hidden_dim]
        """
        # Paso 1: Obtener representaciones globales de la fuente y el destino usando mean pooling con máscara
        if source_mask is not None:
            src_lengths = source_mask.sum(dim=1, keepdim=True).clamp(min=1)
            source_rep = (source_states * source_mask.unsqueeze(2)).sum(dim=1) / src_lengths
        else:
            source_rep = source_states.mean(dim=1)  # [batch, hidden_dim]
        
        if target_mask is not None:
            tgt_lengths = target_mask.sum(dim=1, keepdim=True).clamp(min=1)
            target_rep = (target_states * target_mask.unsqueeze(2)).sum(dim=1) / tgt_lengths
        else:
            target_rep = target_states.mean(dim=1)  # [batch, hidden_dim]
        
        # Aplicar dropout a las representaciones
        source_rep = self.dropout(source_rep)
        target_rep = self.dropout(target_rep)
        
        # Paso 2: Proyectar las representaciones
        source_proj = self.activation(self.attention_source(source_rep))  # [batch, hidden_dim]
        target_proj = self.activation(self.attention_target(target_rep))  # [batch, hidden_dim]
        
        # Paso 3: Atención de fuente hacia destino
        source_proj_expanded = source_proj.unsqueeze(2)  # [batch, hidden_dim, 1]
        attention_weights_target = torch.bmm(target_states, source_proj_expanded).squeeze(2)  # [batch, tgt_len]
        
        if target_mask is not None:
            attention_weights_target = attention_weights_target.masked_fill(target_mask == 0, -1e10)
        
        attention_weights_target = F.softmax(attention_weights_target, dim=1)  # [batch, tgt_len]
        attention_weights_target = self.dropout(attention_weights_target)
        
        # Vector de contexto para el destino
        target_context = torch.bmm(attention_weights_target.unsqueeze(1), target_states).squeeze(1)  # [batch, hidden_dim]
        
        # Paso 4: Atención de destino hacia fuente
        target_proj_expanded = target_proj.unsqueeze(2)  # [batch, hidden_dim, 1]
        attention_weights_source = torch.bmm(source_states, target_proj_expanded).squeeze(2)  # [batch, src_len]
        
        if source_mask is not None:
            attention_weights_source = attention_weights_source.masked_fill(source_mask == 0, -1e10)
        
        attention_weights_source = F.softmax(attention_weights_source, dim=1)  # [batch, src_len]
        attention_weights_source = self.dropout(attention_weights_source)
        
        # Vector de contexto para la fuente
        source_context = torch.bmm(attention_weights_source.unsqueeze(1), source_states).squeeze(1)  # [batch, hidden_dim]
        
        # Paso 5: Concatenar los vectores de contexto
        combined_context = torch.cat([target_context, source_context], dim=1)  # [batch, hidden_dim * 2]
        combined_context = self.dropout(combined_context)
        
        # Paso 6: Pasar por una capa completamente conectada y aplicar activación
        combined_representation = self.activation(self.fc(combined_context))  # [batch, hidden_dim]
        combined_representation = self.dropout(combined_representation)
        
        return combined_representation  # [batch, hidden_dim]

class DynamicCoattentionNetwork(nn.Module):
    """
    Red de Co-Atención Dinámica para tareas de procesamiento de lenguaje natural.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes=3, num_layers=1, dropout=0.3):
        """
        Args:
            vocab_size (int): Tamaño del vocabulario.
            embedding_dim (int): Dimensionalidad de los embeddings.
            hidden_dim (int): Dimensionalidad de los estados ocultos de los LSTMs.
            num_classes (int, opcional): Número de clases para clasificación. Por defecto es 3.
            num_layers (int, opcional): Número de capas en los LSTMs. Por defecto es 1.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.3.
        """
        super(DynamicCoattentionNetwork, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        
        # Embedding layer con manejo de padding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        
        # LSTMs bidireccionales para documento y pregunta
        self.document_lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                                     bidirectional=True, batch_first=True, dropout=dropout if num_layers >1 else 0)
        self.question_lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                                     bidirectional=True, batch_first=True, dropout=dropout if num_layers >1 else 0)
        
        # Mecanismo de Co-Atención
        self.coattention = Coattention(hidden_dim * 2, dropout=dropout)
        
        # LSTM para la salida final (opcional, según la tarea)
        self.coattention_lstm = nn.LSTM(hidden_dim * 2, hidden_dim, 
                                        bidirectional=True, batch_first=True, dropout=dropout)
        
        # Capa completamente conectada para clasificación
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
        
        # Inicialización de pesos
        self._init_weights()
        
    def _init_weights(self):
        """
        Inicializa los pesos de las capas de embedding y lineales utilizando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.coattention.fc.weight)
        nn.init.constant_(self.coattention.fc.bias, 0)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
    
    def forward(self, document, question, doc_lengths, ques_lengths):
        """
        Propagación hacia adelante del modelo Dynamic Co-Attention Network.
    
        Args:
            document (Tensor): Índices de palabras del documento [batch, doc_len]
            question (Tensor): Índices de palabras de la pregunta [batch, ques_len]
            doc_lengths (Tensor): Longitudes de cada documento [batch]
            ques_lengths (Tensor): Longitudes de cada pregunta [batch]
    
        Returns:
            Tensor: Log probabilidades para cada clase [batch, num_classes]
        """
        # Paso 1: Embedding y aplicación de dropout
        doc_embed = self.dropout(self.embedding(document))    # [batch, doc_len, embedding_dim]
        ques_embed = self.dropout(self.embedding(question))   # [batch, ques_len, embedding_dim]
        
        # Paso 2: Pasar por LSTMs con manejo de secuencias
        # Document
        packed_doc = nn.utils.rnn.pack_padded_sequence(doc_embed, doc_lengths.cpu(), 
                                                      batch_first=True, enforce_sorted=False)
        doc_output, _ = self.document_lstm(packed_doc)        # [batch, doc_len, hidden_dim*2]
        doc_output, _ = nn.utils.rnn.pad_packed_sequence(doc_output, batch_first=True)
        
        # Pregunta
        packed_ques = nn.utils.rnn.pack_padded_sequence(ques_embed, ques_lengths.cpu(), 
                                                       batch_first=True, enforce_sorted=False)
        ques_output, _ = self.question_lstm(packed_ques)      # [batch, ques_len, hidden_dim*2]
        ques_output, _ = nn.utils.rnn.pad_packed_sequence(ques_output, batch_first=True)
        
        # Paso 3: Mecanismo de Co-Atención para obtener representación combinada
        # Crear máscaras si las secuencias tienen padding
        source_mask = (document != 0).float()                  # [batch, doc_len]
        target_mask = (question != 0).float()                  # [batch, ques_len]
        
        combined_representation = self.coattention(doc_output, ques_output, 
                                                   source_mask=source_mask, target_mask=target_mask)  # [batch, hidden_dim*2]
        
        # Paso 4: Pasar por otro LSTM para capturar relaciones dinámicas
        # Expandir las dimensiones para LSTM: [batch, 1, hidden_dim*2]
        combined_representation = combined_representation.unsqueeze(1)
        coattention_output, _ = self.coattention_lstm(combined_representation)  # [batch, 1, hidden_dim*2]
        coattention_output = coattention_output.squeeze(1)  # [batch, hidden_dim*2]
        
        # Paso 5: Capa de clasificación
        logits = self.fc(coattention_output)  # [batch, num_classes]
        
        # Retornar log probabilidades
        return F.log_softmax(logits, dim=1)  # [batch, num_classes]

# Configuración del modelo y ejemplo de uso
if __name__ == "__main__":
    # Parámetros del modelo
    vocab_size = 10000    # Tamaño del vocabulario
    embedding_dim = 300   # Dimensionalidad de los embeddings
    hidden_dim = 128      # Dimensionalidad oculta de los LSTMs
    num_classes = 3       # Número de clases para clasificación
    num_layers = 1        # Número de capas en los LSTMs
    dropout = 0.3         # Tasa de dropout
    
    # Crear instancia del modelo
    model = DynamicCoattentionNetwork(vocab_size, embedding_dim, hidden_dim, 
                                      num_classes=num_classes, num_layers=num_layers, dropout=dropout)
    
    # Seleccionar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Ejemplo de datos de entrada
    batch_size = 4
    doc_len = 50
    ques_len = 10
    
    # Simulación de datos de entrada (documento y pregunta)
    # Asumiendo que 0 es el índice de padding
    document = torch.randint(1, vocab_size, (batch_size, doc_len)).to(device)  # [batch, doc_len]
    question = torch.randint(1, vocab_size, (batch_size, ques_len)).to(device)   # [batch, ques_len]
    
    # Longitudes de las secuencias (sin padding)
    doc_lengths = torch.tensor([50, 45, 30, 20]).to(device)   # [batch]
    ques_lengths = torch.tensor([10, 8, 5, 7]).to(device)    # [batch]
    
    # Obtener salida del modelo
    output = model(document, question, doc_lengths, ques_lengths)
    print("Probabilidades de clasificación:", output)
    print("Forma de la salida:", output.shape)  # [batch, num_classes]


### Autoatención

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    """
    Módulo de Self-Attention mejorado con soporte para máscaras, dropout,
    layer normalization y conexiones residuales.
    """
    def __init__(self, embedding_dim, hidden_dim, num_heads=1, dropout=0.1):
        """
        Args:
            embedding_dim (int): Dimensionalidad de los embeddings de entrada.
            hidden_dim (int): Dimensionalidad de las proyecciones de Query, Key y Value.
            num_heads (int, opcional): Número de cabezas de atención. Por defecto es 1.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.1.
        """
        super(SelfAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim debe ser divisible por num_heads"
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        # Proyecciones para Query, Key y Value
        self.query = nn.Linear(embedding_dim, hidden_dim)
        self.key = nn.Linear(embedding_dim, hidden_dim)
        self.value = nn.Linear(embedding_dim, hidden_dim)

        # Dropout para los pesos de atención
        self.dropout = nn.Dropout(dropout)

        # Layer Normalization
        self.layer_norm = nn.LayerNorm(embedding_dim)

        # Proyección final
        self.fc_out = nn.Linear(hidden_dim, embedding_dim)

        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        """
        Inicializa los pesos de las capas lineales utilizando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.query.weight)
        nn.init.xavier_uniform_(self.key.weight)
        nn.init.xavier_uniform_(self.value.weight)
        nn.init.xavier_uniform_(self.fc_out.weight)
        nn.init.zeros_(self.query.bias)
        nn.init.zeros_(self.key.bias)
        nn.init.zeros_(self.value.bias)
        nn.init.zeros_(self.fc_out.bias)

    def forward(self, x, mask=None):
        """
        Propagación hacia adelante del módulo de Self-Attention.

        Args:
            x (Tensor): Entrada de embeddings [batch_size, seq_len, embedding_dim].
            mask (Tensor, opcional): Máscara de atención [batch_size, 1, 1, seq_len] o [batch_size, 1, seq_len].

        Returns:
            Tuple[Tensor, Tensor]: 
                - Output de atención contextualizada [batch_size, seq_len, embedding_dim].
                - Pesos de atención [batch_size, num_heads, seq_len, seq_len].
        """
        batch_size, seq_len, _ = x.size()

        # Aplicar Layer Normalization y Residual Connection
        residual = x
        x = self.layer_norm(x)

        # Proyección y reshape para Multi-Head Attention
        Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, seq_len, head_dim]
        K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)    # [batch, num_heads, seq_len, head_dim]
        V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, seq_len, head_dim]

        # Calcular scores de atención
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # [batch, num_heads, seq_len, seq_len]

        if mask is not None:
            # Asegurarse de que la máscara tiene el mismo número de dimensiones que scores
            # mask debe ser de forma [batch_size, 1, 1, seq_len] o [batch_size, 1, seq_len, seq_len]
            scores = scores.masked_fill(mask == 0, float('-inf'))

        # Aplicar softmax para obtener los pesos de atención
        attention_weights = F.softmax(scores, dim=-1)  # [batch, num_heads, seq_len, seq_len]
        attention_weights = self.dropout(attention_weights)

        # Multiplicar los pesos de atención con los valores
        attention_output = torch.matmul(attention_weights, V)  # [batch, num_heads, seq_len, head_dim]

        # Concatenar las cabezas de atención
        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_dim)  # [batch, seq_len, hidden_dim]

        # Proyección final
        attention_output = self.fc_out(attention_output)  # [batch, seq_len, embedding_dim]
        attention_output = self.dropout(attention_output)

        # Residual Connection
        output = attention_output + residual  # [batch, seq_len, embedding_dim]

        return output, attention_weights

# Ejemplo de uso del módulo SelfAttention mejorado
if __name__ == "__main__":
    # Parámetros de ejemplo
    embedding_dim = 64
    hidden_dim = 64  # Debe ser divisible por num_heads
    num_heads = 8
    dropout = 0.1
    seq_len = 10
    batch_size = 2

    # Instanciar el módulo de Self-Attention
    self_attention = SelfAttention(embedding_dim, hidden_dim, num_heads=num_heads, dropout=dropout)

    # Datos de entrada simulados (batch de secuencias de palabras)
    x = torch.randn(batch_size, seq_len, embedding_dim)  # [batch_size, seq_len, embedding_dim]

    # Ejemplo de máscara (opcional)
    # Supongamos que la primera secuencia tiene padding en las últimas 2 posiciones
    mask = torch.tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    ])  # [batch_size, seq_len]
    mask = mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, seq_len]

    # Obtener salida de atención y matriz de pesos
    output, attention_weights = self_attention(x, mask=mask)

    print("Output de atención:", output.shape)  # [batch_size, seq_len, embedding_dim]
    print("Pesos de atención:", attention_weights.shape)  # [batch_size, num_heads, seq_len, seq_len]


### Key-Value (Predict)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KeyValuePredictAttention(nn.Module):
    """
    Módulo de Atención Key-Value-Predict mejorado con soporte para máscaras, dropout,
    layer normalization y conexiones residuales.
    """
    def __init__(self, embedding_dim, hidden_dim, num_heads=1, dropout=0.1):
        """
        Args:
            embedding_dim (int): Dimensionalidad de los embeddings de entrada.
            hidden_dim (int): Dimensionalidad de las proyecciones de Key, Value y Predict.
            num_heads (int, opcional): Número de cabezas de atención. Por defecto es 1.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.1.
        """
        super(KeyValuePredictAttention, self).__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim debe ser divisible por num_heads"
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads

        # Proyecciones para Key, Value y Predict
        self.key_proj = nn.Linear(embedding_dim, hidden_dim)
        self.value_proj = nn.Linear(embedding_dim, hidden_dim)
        self.predict_proj = nn.Linear(embedding_dim, hidden_dim)

        # Dropout para los pesos de atención
        self.dropout = nn.Dropout(dropout)

        # Layer Normalization
        self.layer_norm = nn.LayerNorm(embedding_dim)

        # Proyección final
        self.fc_out = nn.Linear(hidden_dim * 2, embedding_dim)

        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        """
        Inicializa los pesos de las capas lineales utilizando inicialización Xavier.
        """
        nn.init.xavier_uniform_(self.key_proj.weight)
        nn.init.xavier_uniform_(self.value_proj.weight)
        nn.init.xavier_uniform_(self.predict_proj.weight)
        nn.init.xavier_uniform_(self.fc_out.weight)
        nn.init.zeros_(self.key_proj.bias)
        nn.init.zeros_(self.value_proj.bias)
        nn.init.zeros_(self.predict_proj.bias)
        nn.init.zeros_(self.fc_out.bias)

    def forward(self, x, mask=None):
        """
        Propagación hacia adelante del módulo de Atención Key-Value-Predict.

        Args:
            x (Tensor): Entrada de embeddings [batch_size, seq_len, embedding_dim].
            mask (Tensor, opcional): Máscara de atención [batch_size, 1, 1, seq_len] o [batch_size, 1, seq_len].

        Returns:
            Tuple[Tensor, Tensor]: 
                - Output de atención contextualizada [batch_size, seq_len, embedding_dim].
                - Pesos de atención [batch_size, num_heads, seq_len, seq_len].
        """
        batch_size, seq_len, _ = x.size()

        # Aplicar Layer Normalization y Residual Connection
        residual = x
        x = self.layer_norm(x)

        # Proyección y reshape para Multi-Head Attention
        K = self.key_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)    # [batch, num_heads, seq_len, head_dim]
        V = self.value_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, seq_len, head_dim]
        P = self.predict_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)  # [batch, num_heads, seq_len, head_dim]

        # Calcular scores de atención usando Key
        scores = torch.matmul(K, K.transpose(-2, -1)) / (self.head_dim ** 0.5)  # [batch, num_heads, seq_len, seq_len]

        if mask is not None:
            # Asegurarse de que la máscara tiene el mismo número de dimensiones que scores
            # mask debe ser de forma [batch_size, 1, 1, seq_len] o [batch_size, 1, seq_len, seq_len]
            scores = scores.masked_fill(mask == 0, float('-inf'))

        # Aplicar softmax para obtener los pesos de atención
        attention_weights = F.softmax(scores, dim=-1)  # [batch, num_heads, seq_len, seq_len]
        attention_weights = self.dropout(attention_weights)

        # Multiplicar los pesos de atención con Value
        context = torch.matmul(attention_weights, V)  # [batch, num_heads, seq_len, head_dim]

        # Multiplicar los pesos de atención con Predict
        predict = torch.matmul(attention_weights, P)  # [batch, num_heads, seq_len, head_dim]

        # Concatenar Contexto y Predict
        combined = torch.cat([context, predict], dim=-1)  # [batch, num_heads, seq_len, head_dim * 2]

        # Proyección final
        combined = combined.view(batch_size, self.num_heads, seq_len, self.head_dim * 2)
        combined = combined.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_dim * 2)  # [batch, seq_len, hidden_dim * 2]
        output = self.fc_out(combined)  # [batch, seq_len, embedding_dim]
        output = self.dropout(output)

        # Residual Connection
        output = output + residual  # [batch, seq_len, embedding_dim]

        return output, attention_weights

# Ejemplo de uso del módulo KeyValuePredictAttention mejorado
if __name__ == "__main__":
    # Parámetros de ejemplo
    embedding_dim = 64
    hidden_dim = 64  # Debe ser divisible por num_heads
    num_heads = 8
    dropout = 0.1
    seq_len = 10
    batch_size = 2

    # Instanciar el módulo de Key-Value-Predict Attention
    kvp_attention = KeyValuePredictAttention(embedding_dim, hidden_dim, num_heads=num_heads, dropout=dropout)

    # Mover el modelo al dispositivo adecuado
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    kvp_attention.to(device)

    # Datos de entrada simulados (batch de secuencias de palabras)
    x = torch.randn(batch_size, seq_len, embedding_dim).to(device)  # [batch_size, seq_len, embedding_dim]

    # Ejemplo de máscara (opcional)
    # Supongamos que la primera secuencia tiene padding en las últimas 2 posiciones
    mask = torch.tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    ]).to(device)  # [batch_size, seq_len]
    mask = mask.unsqueeze(1).unsqueeze(2)  # [batch_size, 1, 1, seq_len]

    # Obtener salida de atención y matriz de pesos
    output, attention_weights = kvp_attention(x, mask=mask)

    print("Output de atención:", output.shape)  # [batch_size, seq_len, embedding_dim]
    print("Pesos de atención:", attention_weights.shape)  # [batch_size, num_heads, seq_len, seq_len]


### Atención sobre atención

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionOverAttention(nn.Module):
    """
    Módulo de Atención sobre Atención (Attention over Attention) mejorado.
    
    Este módulo realiza una atención bidireccional entre un documento y una pregunta,
    combinando las representaciones contextuales para producir un vector de contexto final.
    
    Mejoras:
        - Soporte para máscaras de padding.
        - Incorporación de Dropout para regularización.
        - Layer Normalization para estabilidad del entrenamiento.
        - Conexiones residuales para facilitar el flujo de gradientes.
        - Inicialización adecuada de pesos.
        - Configurabilidad de parámetros como número de capas GRU y tasa de dropout.
    """
    def __init__(self, embedding_dim, hidden_dim, num_layers=1, dropout=0.1):
        """
        Inicializa el módulo AttentionOverAttention.
        
        Args:
            embedding_dim (int): Dimensionalidad de los embeddings de entrada.
            hidden_dim (int): Dimensionalidad de los estados ocultos de las Bi-GRU.
            num_layers (int, opcional): Número de capas en las Bi-GRU. Por defecto es 1.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.1.
        """
        super(AttentionOverAttention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout_rate = dropout
        
        # Capas Bi-GRU para documento y pregunta
        self.document_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.question_gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Layer Normalization
        self.layer_norm = nn.LayerNorm(hidden_dim * 2)
        
        # Proyección final
        self.fc = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        
        # Inicialización de pesos
        self._init_weights()
        
    def _init_weights(self):
        """
        Inicializa los pesos de las capas lineales y GRUs utilizando inicialización Xavier.
        """
        for name, param in self.named_parameters():
            if 'weight' in name and isinstance(param, nn.Linear):
                nn.init.xavier_uniform_(param)
            elif 'bias' in name and isinstance(param, nn.Linear):
                nn.init.zeros_(param)
            elif 'weight_ih' in name and isinstance(param, nn.GRU):
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name and isinstance(param, nn.GRU):
                nn.init.xavier_uniform_(param.data)
            elif 'bias_ih' in name and isinstance(param, nn.GRU):
                nn.init.zeros_(param.data)
            elif 'bias_hh' in name and isinstance(param, nn.GRU):
                nn.init.zeros_(param.data)
    
    def forward(self, document, question, doc_lengths=None, ques_lengths=None):
        """
        Propagación hacia adelante del módulo AttentionOverAttention.
        
        Args:
            document (Tensor): Embeddings del documento [batch_size, doc_len, embedding_dim].
            question (Tensor): Embeddings de la pregunta [batch_size, ques_len, embedding_dim].
            doc_lengths (Tensor, opcional): Longitudes reales de cada documento [batch_size].
            ques_lengths (Tensor, opcional): Longitudes reales de cada pregunta [batch_size].
        
        Returns:
            Tuple[Tensor, Tensor]:
                - Context Vector: Representación combinada [batch_size, hidden_dim*2].
                - Attention-over-Attention: Matriz de atención [batch_size, doc_len].
        """
        # Paso 1: Pasar el documento y la pregunta por las Bi-GRUs
        doc_output, _ = self._run_gru(self.document_gru, document, doc_lengths)  # [batch, doc_len, hidden_dim*2]
        ques_output, _ = self._run_gru(self.question_gru, question, ques_lengths)  # [batch, ques_len, hidden_dim*2]
        
        # Paso 2: Calcular la matriz de atención mediante el producto punto
        attention_matrix = torch.bmm(doc_output, ques_output.transpose(1, 2))  # [batch, doc_len, ques_len]
        
        # Paso 3: Aplicar Softmax en ambas direcciones
        # Softmax por columnas (atención del documento hacia la pregunta)
        column_softmax = F.softmax(attention_matrix, dim=2)  # [batch, doc_len, ques_len]
        
        # Softmax por filas (atención de la pregunta hacia el documento)
        row_softmax = F.softmax(attention_matrix.transpose(1, 2), dim=2)  # [batch, ques_len, doc_len]
        
        # Paso 4: Calcular Atención sobre Atención (AoA)
        # Promediar la atención en las columnas para obtener un vector de atención final
        attention_over_attention = torch.mean(column_softmax * row_softmax.transpose(1, 2), dim=2)  # [batch, doc_len]
        
        # Paso 5: Ponderar el documento con AoA
        # Expandir la dimensión para multiplicar y obtener una representación final
        context_vector = torch.bmm(attention_over_attention.unsqueeze(1), doc_output).squeeze(1)  # [batch, hidden_dim*2]
        
        # Paso 6: Aplicar Layer Normalization y Dropout
        context_vector = self.layer_norm(context_vector)
        context_vector = self.dropout(context_vector)
        
        # Paso 7: Pasar por una capa completamente conectada
        context_vector = F.relu(self.fc(context_vector))  # [batch, hidden_dim*2]
        context_vector = self.dropout(context_vector)
        
        return context_vector, attention_over_attention
    
    def _run_gru(self, gru, x, lengths):
        """
        Ejecuta una GRU bidireccional con manejo de secuencias empaquetadas.
        
        Args:
            gru (nn.GRU): La capa GRU a utilizar.
            x (Tensor): Entrada a la GRU [batch_size, seq_len, embedding_dim].
            lengths (Tensor, opcional): Longitudes reales de las secuencias [batch_size].
        
        Returns:
            Tuple[Tensor, Tensor]:
                - Output de la GRU [batch_size, seq_len, hidden_dim*2].
                - Estado oculto final.
        """
        if lengths is not None:
            # Ordenar las secuencias por longitud descendente
            lengths_sorted, sorted_idx = lengths.sort(descending=True)
            x_sorted = x[sorted_idx]
            
            # Empaquetar las secuencias
            packed_input = nn.utils.rnn.pack_padded_sequence(x_sorted, lengths_sorted.cpu(), batch_first=True)
            
            # Pasar por la GRU
            packed_output, hidden = gru(packed_input)
            
            # Desempaquetar las secuencias
            output_sorted, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
            
            # Restaurar el orden original
            _, original_idx = sorted_idx.sort(0, descending=False)
            output = output_sorted[original_idx]
            hidden = hidden[:, original_idx]
        else:
            output, hidden = gru(x)
        
        return output, hidden

# Ejemplo de uso del módulo AttentionOverAttention mejorado
if __name__ == "__main__":
    # Parámetros de ejemplo
    embedding_dim = 128
    hidden_dim = 64
    num_layers = 2
    dropout = 0.3
    batch_size = 2
    doc_len = 10
    ques_len = 5

    # Crear instancia del modelo Attention-over-Attention
    aoa_model = AttentionOverAttention(embedding_dim, hidden_dim, num_layers=num_layers, dropout=dropout)

    # Mover el modelo al dispositivo adecuado
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    aoa_model.to(device)

    # Simulación de datos de entrada (documento y pregunta)
    document = torch.randn(batch_size, doc_len, embedding_dim).to(device)   # [batch, doc_len, embedding_dim]
    question = torch.randn(batch_size, ques_len, embedding_dim).to(device)  # [batch, ques_len, embedding_dim]

    # Simulación de longitudes reales (sin padding)
    doc_lengths = torch.tensor([10, 7]).to(device)  # [batch]
    ques_lengths = torch.tensor([5, 3]).to(device)  # [batch]

    # Obtener salida del modelo
    context_vector, attention_over_attention = aoa_model(document, question, doc_lengths, ques_lengths)

    print("Vector de contexto:", context_vector.shape)  # [batch, hidden_dim*2]
    print("Attention-over-Attention:", attention_over_attention.shape)  # [batch, doc_len]


### Modelo de flujo de atención

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AttentionFlow(nn.Module):
    """
    Módulo de Atención Flow para el modelo BiDAF.

    Este módulo implementa el mecanismo de atención bidireccional entre el contexto y la pregunta,
    calculando las atenciones Query2Context y Context2Query.
    """
    def __init__(self, hidden_dim):
        """
        Inicializa el módulo AttentionFlow.

        Args:
            hidden_dim (int): Dimensionalidad de las representaciones ocultas.
        """
        super(AttentionFlow, self).__init__()
        self.hidden_dim = hidden_dim

        # Parámetros para calcular la similitud
        self.W_c = nn.Linear(hidden_dim * 2, 1, bias=False)
        self.W_q = nn.Linear(hidden_dim * 2, 1, bias=False)
        self.W_cq = nn.Linear(hidden_dim * 2, 1, bias=False)

    def forward(self, context, question, context_mask, question_mask):
        """
        Propagación hacia adelante del módulo AttentionFlow.

        Args:
            context (Tensor): Representaciones del contexto [batch_size, context_len, hidden_dim * 2].
            question (Tensor): Representaciones de la pregunta [batch_size, question_len, hidden_dim * 2].
            context_mask (Tensor): Máscara del contexto [batch_size, context_len].
            question_mask (Tensor): Máscara de la pregunta [batch_size, question_len].

        Returns:
            Tensor: Representaciones combinadas [batch_size, context_len, hidden_dim * 8].
        """
        batch_size, context_len, _ = context.size()
        question_len = question.size(1)

        # Expandir dimensiones para calcular la similitud
        context_exp = context.unsqueeze(2).repeat(1, 1, question_len, 1)   # [batch, context_len, question_len, hidden_dim * 2]
        question_exp = question.unsqueeze(1).repeat(1, context_len, 1, 1)  # [batch, context_len, question_len, hidden_dim * 2]

        # Calcular la matriz de similitud
        S = self._similarity_matrix(context_exp, question_exp)  # [batch, context_len, question_len]

        # Aplicar máscaras para ignorar posiciones de padding
        question_mask = question_mask.unsqueeze(1).expand(-1, context_len, -1)  # [batch, context_len, question_len]
        S = S.masked_fill(~question_mask, float('-inf'))

        # Atención Contexto a Pregunta (C2Q)
        a = F.softmax(S, dim=-1)  # [batch, context_len, question_len]
        c2q = torch.bmm(a, question)  # [batch, context_len, hidden_dim * 2]

        # Atención Pregunta a Contexto (Q2C)
        b = F.softmax(S.max(dim=2)[0], dim=-1).unsqueeze(1)  # [batch, 1, context_len]
        q2c = torch.bmm(b, context).repeat(1, context_len, 1)  # [batch, context_len, hidden_dim * 2]

        # Concatenar representaciones
        G = torch.cat([context, c2q, context * c2q, context * q2c], dim=-1)  # [batch, context_len, hidden_dim * 8]

        return G

    def _similarity_matrix(self, context_exp, question_exp):
        """
        Calcula la matriz de similitud entre el contexto y la pregunta.

        Args:
            context_exp (Tensor): Contexto expandido [batch, context_len, question_len, hidden_dim * 2].
            question_exp (Tensor): Pregunta expandida [batch, context_len, question_len, hidden_dim * 2].

        Returns:
            Tensor: Matriz de similitud [batch, context_len, question_len].
        """
        S = self.W_c(context_exp).squeeze(-1) + self.W_q(question_exp).squeeze(-1) + \
            self.W_cq(context_exp * question_exp).squeeze(-1)
        return S

class ModelingLayer(nn.Module):
    """
    Módulo de Modeling Layer para el modelo BiDAF.

    Este módulo utiliza una Bi-GRU para modelar las relaciones dinámicas entre las palabras en el contexto.
    """
    def __init__(self, hidden_dim, num_layers=2, dropout=0.2):
        """
        Inicializa el módulo ModelingLayer.

        Args:
            hidden_dim (int): Dimensionalidad de las representaciones ocultas.
            num_layers (int, opcional): Número de capas en la Bi-GRU. Por defecto es 2.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.2.
        """
        super(ModelingLayer, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.gru = nn.GRU(
            input_size=hidden_dim * 8,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )

    def forward(self, G):
        """
        Propagación hacia adelante del módulo ModelingLayer.

        Args:
            G (Tensor): Representaciones combinadas de la capa de atención [batch_size, context_len, hidden_dim * 8].

        Returns:
            Tensor: Representaciones modeladas [batch_size, context_len, hidden_dim * 2].
        """
        M, _ = self.gru(G)  # [batch_size, context_len, hidden_dim * 2]
        return M

class OutputLayer(nn.Module):
    """
    Módulo de Output Layer para el modelo BiDAF.

    Este módulo predice las posiciones de inicio y fin de la respuesta en el contexto.
    """
    def __init__(self, hidden_dim, dropout=0.2):
        """
        Inicializa el módulo OutputLayer.

        Args:
            hidden_dim (int): Dimensionalidad de las representaciones ocultas.
            dropout (float, opcional): Tasa de dropout. Por defecto es 0.2.
        """
        super(OutputLayer, self).__init__()
        self.hidden_dim = hidden_dim

        self.p1_weight_g = nn.Linear(hidden_dim * 8, 1)
        self.p1_weight_m = nn.Linear(hidden_dim * 2, 1)
        self.p2_weight_g = nn.Linear(hidden_dim * 8, 1)
        self.p2_weight_m = nn.Linear(hidden_dim * 2, 1)

        self.rnn = nn.GRU(
            input_size=hidden_dim * 2,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, G, M, mask):
        """
        Propagación hacia adelante del módulo OutputLayer.

        Args:
            G (Tensor): Representaciones combinadas de la capa de atención [batch_size, context_len, hidden_dim * 8].
            M (Tensor): Representaciones modeladas [batch_size, context_len, hidden_dim * 2].
            mask (Tensor): Máscara del contexto [batch_size, context_len].

        Returns:
            Tuple[Tensor, Tensor]: Logits de inicio y fin [batch_size, context_len].
        """
        # Predicción de inicio
        logits1 = (self.p1_weight_g(G) + self.p1_weight_m(M)).squeeze(-1)  # [batch_size, context_len]
        logits1 = logits1.masked_fill(~mask, float('-inf'))

        # Pasar M por otra capa GRU
        M2, _ = self.rnn(M)  # [batch_size, context_len, hidden_dim * 2]

        # Predicción de fin
        logits2 = (self.p2_weight_g(G) + self.p2_weight_m(M2)).squeeze(-1)  # [batch_size, context_len]
        logits2 = logits2.masked_fill(~mask, float('-inf'))

        return logits1, logits2

class BiDAF(nn.Module):
    """
    Modelo BiDAF (Bidirectional Attention Flow) para tareas de preguntas y respuestas.

    Este modelo sigue la arquitectura presentada en el artículo original de BiDAF,
    implementando capas de embedding contextual, atención bidireccional, modelado y capa de salida.
    """
    def __init__(self, word_dim, char_dim, hidden_dim, vocab_size, char_vocab_size, embedding_matrix=None):
        """
        Inicializa el modelo BiDAF.

        Args:
            word_dim (int): Dimensionalidad de los embeddings de palabra.
            char_dim (int): Dimensionalidad de los embeddings de carácter.
            hidden_dim (int): Dimensionalidad de las representaciones ocultas en las GRUs.
            vocab_size (int): Tamaño del vocabulario de palabras.
            char_vocab_size (int): Tamaño del vocabulario de caracteres.
            embedding_matrix (Tensor, opcional): Matriz de embeddings preentrenados. Por defecto es None.
        """
        super(BiDAF, self).__init__()
        self.hidden_dim = hidden_dim

        # Embedding de palabras
        self.word_embedding = nn.Embedding(vocab_size, word_dim, padding_idx=0)
        if embedding_matrix is not None:
            self.word_embedding.weight.data.copy_(embedding_matrix)

        # Embedding de caracteres
        self.char_embedding = nn.Embedding(char_vocab_size, char_dim, padding_idx=0)
        self.char_conv = nn.Conv1d(in_channels=char_dim, out_channels=char_dim, kernel_size=5, padding=2)

        # Contextual Embedding Layer (Bi-GRU)
        self.contextual_gru = nn.GRU(
            input_size=word_dim + char_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        # Attention Flow Layer
        self.attention_flow = AttentionFlow(hidden_dim)

        # Modeling Layer
        self.modeling_layer = ModelingLayer(hidden_dim)

        # Output Layer
        self.output_layer = OutputLayer(hidden_dim)

        # Inicialización de pesos
        self._init_weights()

    def _init_weights(self):
        """
        Inicializa los pesos de las capas lineales y convolucionales.
        """
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

    def forward(self, context, query, context_char=None, query_char=None):
        """
        Propagación hacia adelante del modelo BiDAF.

        Args:
            context (Tensor): Índices de palabras del contexto [batch_size, context_len].
            query (Tensor): Índices de palabras de la pregunta [batch_size, query_len].
            context_char (Tensor, opcional): Índices de caracteres del contexto [batch_size, context_len, char_len]. Por defecto es None.
            query_char (Tensor, opcional): Índices de caracteres de la pregunta [batch_size, query_len, char_len]. Por defecto es None.

        Returns:
            Tuple[Tensor, Tensor]:
                - Logits de inicio [batch_size, context_len].
                - Logits de fin [batch_size, context_len].
        """
        batch_size, context_len = context.size()
        query_len = query.size(1)

        # Embedding de palabras
        word_emb_context = self.word_embedding(context)  # [batch_size, context_len, word_dim]
        word_emb_query = self.word_embedding(query)      # [batch_size, query_len, word_dim]

        if context_char is not None and query_char is not None:
            # Embedding de caracteres para el contexto
            char_emb_context = self.char_embedding(context_char)  # [batch_size, context_len, char_len, char_dim]
            batch_size, context_len, char_len, char_dim = char_emb_context.size()
            char_emb_context = char_emb_context.view(-1, char_len, char_dim)  # [batch_size * context_len, char_len, char_dim]
            char_emb_context = char_emb_context.transpose(1, 2)               # [batch_size * context_len, char_dim, char_len]
            char_features_context = F.relu(self.char_conv(char_emb_context))  # [batch_size * context_len, char_dim, char_len]
            char_features_context, _ = torch.max(char_features_context, dim=2)  # [batch_size * context_len, char_dim]
            char_features_context = char_features_context.view(batch_size, context_len, -1)  # [batch_size, context_len, char_dim]

            # Embedding de caracteres para la pregunta
            char_emb_query = self.char_embedding(query_char)  # [batch_size, query_len, char_len, char_dim]
            batch_size, query_len, char_len, char_dim = char_emb_query.size()
            char_emb_query = char_emb_query.view(-1, char_len, char_dim)  # [batch_size * query_len, char_len, char_dim]
            char_emb_query = char_emb_query.transpose(1, 2)               # [batch_size * query_len, char_dim, char_len]
            char_features_query = F.relu(self.char_conv(char_emb_query))  # [batch_size * query_len, char_dim, char_len]
            char_features_query, _ = torch.max(char_features_query, dim=2)  # [batch_size * query_len, char_dim]
            char_features_query = char_features_query.view(batch_size, query_len, -1)  # [batch_size, query_len, char_dim]

            # Concatenar embeddings de palabras y caracteres
            word_emb_context = torch.cat([word_emb_context, char_features_context], dim=-1)  # [batch_size, context_len, word_dim + char_dim]
            word_emb_query = torch.cat([word_emb_query, char_features_query], dim=-1)        # [batch_size, query_len, word_dim + char_dim]

        # Contextual Embedding Layer
        context_output, _ = self.contextual_gru(word_emb_context)  # [batch_size, context_len, hidden_dim * 2]
        query_output, _ = self.contextual_gru(word_emb_query)      # [batch_size, query_len, hidden_dim * 2]

        # Máscaras
        context_mask = (context != 0)  # [batch_size, context_len]
        question_mask = (query != 0)   # [batch_size, query_len]

        # Attention Flow Layer
        G = self.attention_flow(context_output, query_output, context_mask, question_mask)  # [batch_size, context_len, hidden_dim * 8]

        # Modeling Layer
        M = self.modeling_layer(G)  # [batch_size, context_len, hidden_dim * 2]

        # Output Layer
        start_logits, end_logits = self.output_layer(G, M, context_mask)  # [batch_size, context_len], [batch_size, context_len]

        return start_logits, end_logits

# Configuración de ejemplo
if __name__ == "__main__":
    # Parámetros del modelo
    word_dim = 100       # Dimensionalidad del embedding de palabra
    char_dim = 50        # Dimensionalidad del embedding de carácter
    hidden_dim = 64      # Dimensionalidad oculta de las GRUs
    vocab_size = 10000   # Tamaño del vocabulario de palabras
    char_vocab_size = 100  # Tamaño del vocabulario de caracteres
    batch_size = 2
    context_len = 20
    query_len = 10
    char_len = 10        # Longitud de secuencia de caracteres

    # Simulación de matriz de embeddings preentrenados (opcional)
    embedding_matrix = torch.randn(vocab_size, word_dim)

    # Crear instancia del modelo BiDAF
    bidaf_model = BiDAF(word_dim, char_dim, hidden_dim, vocab_size, char_vocab_size, embedding_matrix=embedding_matrix)

    # Seleccionar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bidaf_model.to(device)

    # Simulación de datos de entrada (contexto y pregunta)
    context = torch.randint(1, vocab_size, (batch_size, context_len)).to(device)      # [batch_size, context_len]
    query = torch.randint(1, vocab_size, (batch_size, query_len)).to(device)          # [batch_size, query_len]

    # Simulación de datos de caracteres
    context_char = torch.randint(1, char_vocab_size, (batch_size, context_len, char_len)).to(device)  # [batch_size, context_len, char_len]
    query_char = torch.randint(1, char_vocab_size, (batch_size, query_len, char_len)).to(device)      # [batch_size, query_len, char_len]

    # Obtener salida del modelo
    start_logits, end_logits = bidaf_model(context, query, context_char, query_char)

    print("Logits de inicio:", start_logits.shape)  # [batch_size, context_len]
    print("Logits de fin:", end_logits.shape)       # [batch_size, context_len]


### **Ejercicios sobre mecanismos de atención**

#### **1. Self-Attention**

**a. Implementación de Self-Attention Multi-cabecera**

*Objetivo:* Modificar la implementación de `SelfAttention` para soportar atención multi-cabecera de manera más eficiente, siguiendo la arquitectura utilizada en Transformers.

*Instrucciones:*
- Crea una clase `MultiHeadSelfAttention` que extienda `nn.Module`.
- Asegúrate de que las proyecciones de Query, Key y Value se realicen de manera eficiente para múltiples cabezas.
- Implementa el mecanismo de concatenación y proyección final.
- Añade una capa de *dropout* y *layer normalization* después de la atención.
- Compara tu implementación con la clase `SelfAttention` mejorada proporcionada anteriormente.

*Recursos:*
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- [PyTorch MultiheadAttention](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)

*Desafío Adicional:*
- Añade la posibilidad de enmascarar ciertas posiciones en la atención (por ejemplo, para atención causal en modelos de generación).


**b. Visualización de los pesos de atención**

*Objetivo:* Visualizar los pesos de atención aprendidos por el módulo `SelfAttention` para comprender cómo el modelo asigna importancia a diferentes posiciones en la secuencia.

*Instrucciones:*
- Utiliza una secuencia de entrada sencilla (por ejemplo, una oración corta) y pásala a través del módulo `SelfAttention`.
- Extrae los pesos de atención y utiliza una biblioteca de visualización (como `matplotlib`) para representar gráficamente la matriz de atención.
- Analiza cómo los diferentes tokens se relacionan entre sí según los pesos de atención.




#### **2. KeyValuePredictAttention**

**a. Extensión para soportar máscaras dinámicas**

*Objetivo:* Mejorar la clase `KeyValuePredictAttention` para manejar máscaras dinámicas que permitan trabajar con secuencias de diferentes longitudes en un solo batch.

*Instrucciones:*
- Modifica el método `forward` para aceptar una máscara opcional que indique las posiciones válidas en la secuencia.
- Asegúrate de que la máscara se aplique correctamente al calcular los scores de atención.
- Prueba tu implementación con secuencias de diferentes longitudes y verifica que las posiciones de padding no influyan en la atención.

*Pista:*
- Puedes utilizar `torch.masked_fill` para aplicar la máscara a los scores de atención antes de la softmax.


**b. Integración con una capa de salida personalizada**

*Objetivo:* Añadir una capa de salida personalizada que permita transformar el output de la atención contextualizada para una tarea específica, como clasificación o regresión.

*Instrucciones:*
- Después de obtener el `output` de atención contextualizada, pásalo por una o más capas lineales con activaciones no lineales.
- Implementa una función de pérdida adecuada según la tarea (por ejemplo, `nn.CrossEntropyLoss` para clasificación).
- Entrena el módulo en un conjunto de datos simulado para verificar que la arquitectura es funcional.

*Desafío adicional:*
- Añade una capa de normalización adicional (como `BatchNorm1d`) antes de la capa de salida para mejorar la estabilidad del entrenamiento.


#### **3. AttentionOverAttention**

**a. Implementación de atención bidireccional completa**

*Objetivo:* Completar la implementación de `AttentionOverAttention` para que capture completamente las interacciones bidireccionales entre el contexto y la pregunta.

*Instrucciones:*
- Asegúrate de que tanto la atención de `Query2Context` como `Context2Query` están correctamente implementadas y se combinan adecuadamente.
- Modifica el mecanismo de `AttentionOverAttention` para incluir multiplicaciones element-wise adicionales o interacciones que puedan enriquecer la representación final.
- Evalúa la efectividad de las representaciones resultantes en una tarea de preguntas y respuestas simple.

---

**b. Incorporación de positional encoding**

*Objetivo:* Añadir codificaciones posicionales a las entradas de `AttentionOverAttention` para proporcionar información sobre la posición de cada token en la secuencia.

*Instrucciones:*
- Implementa una clase `PositionalEncoding` que añada codificaciones posicionales sinusoidales a los embeddings.
- Integra esta clase en el flujo de `BiDAF` antes de pasar los embeddings a las GRUs.
- Verifica que las codificaciones posicionales mejoran la capacidad del modelo para capturar relaciones de orden en la secuencia.

*Recursos:*
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)



#### **4. BiDAF**

**a. Mejorar la integración de embeddings de caracteres**

*Objetivo:* Refinar la forma en que los embeddings de caracteres se integran con los embeddings de palabras para mejorar la representación final.

*Instrucciones:*
- En lugar de utilizar una simple convolución y pooling sobre los caracteres, implementa un `Bi-GRU` o una red convolucional más profunda para procesar los embeddings de caracteres.
- Concatenar las representaciones de caracteres procesadas con los embeddings de palabras antes de pasarlos a las GRUs contextuales.
- Evalúa el impacto de esta mejora en la calidad de las representaciones aprendidas y en el rendimiento del modelo.

*Recursos:*
- [Bidirectional Attention Flow for Machine Comprehension](https://arxiv.org/abs/1611.01603)


**b. Implementación de conexiones residuales en BiDAF**

*Objetivo:* Añadir conexiones residuales en las diferentes capas del modelo BiDAF para facilitar el flujo de gradientes y mejorar la capacidad de representación.

*Instrucciones:*
- Añade conexiones residuales después de la capa de `AttentionFlow` y después de la `ModelingLayer`.
- Asegúrate de que las dimensiones coincidan al sumar las salidas y las entradas de las capas.
- Verifica que la adición de conexiones residuales no introduzca errores dimensionales y que el entrenamiento sea más estable.

*Pista:*
- Puedes utilizar `nn.LayerNorm` y `nn.Dropout` antes de las conexiones residuales para mejorar la estabilidad.


**c. Evaluación del modelo BiDAF en un conjunto de datos simulado**

*Objetivo:* Crear un conjunto de datos simulado y entrenar el modelo BiDAF mejorado para una tarea de preguntas y respuestas, evaluando su capacidad para predecir correctamente las posiciones de inicio y fin.

*Instrucciones:*
- Genera datos simulados donde el contexto contiene una respuesta clara a una pregunta dada.
- Define las posiciones de inicio y fin en el contexto que corresponden a la respuesta.
- Entrena el modelo BiDAF en estos datos y observa si aprende a predecir correctamente las posiciones de inicio y fin.
- Ajusta hiperparámetros como la tasa de aprendizaje, el número de épocas y la tasa de dropout para optimizar el rendimiento.

*Desafío adicional:*
- Implementa métricas de evaluación como Exact Match (EM) y F1 para medir la precisión de las predicciones del modelo.





In [ ]:
### Tus respuestas